In [94]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from __future__ import division

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import lal
from lal import CachedDetectors
from lal import LALDetectorIndexLHODIFF,LALDetectorIndexLLODIFF,LALDetectorIndexVIRGODIFF

from lal import ComputeDetAMResponse

import lalsimulation

DETECTOR_SITES = {
    'H1': LALDetectorIndexLHODIFF,
    'L1': LALDetectorIndexLLODIFF,
    'V1': LALDetectorIndexVIRGODIFF
    }



In [95]:
# Calculate first the diagonal elements of kappa with the fitted PSDs of the data 
freqs = np.arange(20000)/10. + 20

#analytic fits of the PSDs
S_L1 = (0.8e-22*(15./(0.08+freqs))**2)**2+0.67e-23**2+((freqs/2000.)*3.e-23)**2
S_H1 = (1e-22*(24./(0.1+freqs))**2)**2+0.7e-23**2+((freqs/2000.)*3.4e-23)**2
S_V1 = (1e-22*(35./(0.1+freqs))**2)**2+0.9e-23**2+((freqs/2000.)*15e-23)**2

integral_nom = np.zeros([len(freqs),3])
integral_nom[:,0] = freqs**(-7./3.)*S_H1**(-1.)
integral_nom[:,1] = freqs**(-7./3.)*S_L1**(-1.)
integral_nom[:,2] = freqs**(-7./3.)*S_V1**(-1.)

integral_denom = freqs**(-7/3)*(S_H1**(-1.)+S_L1**(-1.)+S_V1**(-1.))/3.
                                
kappa = np.zeros(3)

for i in range(3):
    kappa[i] = (integral_nom[:,i].sum())/(integral_denom.sum())
"""
print kappa.sum()
print kappa[0]
print kappa[1]
print kappa[2]
""";

In [96]:
class Detector(object):
    """
    A Detector object characterises a gravitational wave (GW) interferometric detector
    """

    def __init__(self, detector):
        """
        detector  -- label string of the detector
        descriptor -- LAL descriptor
        location -- geographic location of the detector
        response -- response matrix

        """
        self.name = detector
        self.descriptor =  CachedDetectors[DETECTOR_SITES[detector]]
        self.location = lalsimulation.DetectorPrefixToLALDetector(detector).location
        self.response = lalsimulation.DetectorPrefixToLALDetector(detector).response
        
    def antenna_pattern(self, time_at_coalescence, RA, dec):
        """ Compute antenna response
        """
        
        gmst_rad = lal.GreenwichMeanSiderealTime(time_at_coalescence)
        fplus = []
        fcross = []
        for (RA_val, dec_val) in zip(RA, dec):
            fplus_val,fcross_val = ComputeDetAMResponse(self.descriptor.response,
                                    RA_val, dec_val, psi, gmst_rad)
            fplus.append(fplus_val)
            fcross.append(fcross_val)
        return np.array(fplus), np.array(fcross)

In [97]:
#map resolution
n = 1000

ra_plot = np.linspace(-np.pi, np.pi, n)
dec_plot= np.linspace(-np.pi/2, np.pi/2, n)

time_GW170817 = 1187008882.43

time_0 = 1187007040.
iota = np.float(170)
psi = np.float(0)

time = time_0

#Generate Grid for the calculation
ra, dec = np.meshgrid(ra_plot,dec_plot)

for i in range(3):    
    if(i==0):
        detector = Detector('H1')
    if(i==1):
        detector = Detector('L1')
    if(i==2):
        detector = Detector('V1')

    Fplus,Fcross = detector.antenna_pattern(time, ra.flatten(), dec.flatten())
    
    if(i==0):
        FplusH1 = Fplus.reshape(ra.shape)
        FcrossH1 = Fcross.reshape(ra.shape)
    if(i==1):
        FplusL1 = Fplus.reshape(ra.shape)
        FcrossL1 = Fcross.reshape(ra.shape)
    if(i==2):
        FplusV1 = Fplus.reshape(ra.shape)
        FcrossV1 = Fcross.reshape(ra.shape)
    

In [ ]:
from numpy import linalg as LA
from numpy import unravel_index
import matplotlib.cm as cm 

In [ ]:
Theta = np.zeros([2,2,n,n]) 
Theta_diag = np.zeros([n,n])
epsilon = np.zeros([n,n])
index_phi = np.arange(n)
index_theta = np.arange(n)

for i in index_phi:
    for j in index_theta:
        Theta[0,0,j,i] = kappa[0]*FplusH1[j,i]**2 + kappa[1]*FplusL1[j,i]**2 + kappa[2]*FplusV1[j,i]**2
        Theta[0,1,j,i] = kappa[0]*FplusH1[j,i]*FcrossH1[j,i] + kappa[1]*FplusL1[j,i]*FcrossL1[j,i] + \
                        + kappa[2]*FplusV1[j,i]*FcrossV1[j,i]
        Theta[1,0,j,i] = Theta[0,1,j,i]
        Theta[1,1,j,i] = kappa[0]*FcrossH1[j,i]**2 + kappa[1]*FcrossL1[j,i]**2 + kappa[2]*FcrossV1[j,i]**2
        
        #calculate the eigenvalues
        e,_ = LA.eig(np.array(Theta[:,:,j,i]))
        epsilon[j,i] = (e[0]-e[1])/(e[0] + e[1])
        
#identify where the detector network has the highest resolution 
epsilon_abs = abs(epsilon)

ind = unravel_index(epsilon_abs.argmin(), FplusH1.shape)
print ind
ra_plot = np.linspace(-np.pi, np.pi, n)
dec_plot= np.linspace(-np.pi/2, np.pi/2, n)
ra_mesh, dec_mesh = np.meshgrid(ra_plot,dec_plot)
print('ra = {0} deg, dec = {1} deg'.format(ra_plot[ind[1]]*180/np.pi, 180/np.pi*dec_plot[ind[0]]))
print('ra = {0}, dec = {1}'.format(ra_plot[ind[1]],dec_plot[ind[0]]))


In [ ]:
#Calculate ra for different times
time_offset = 100
ra = -0.20776063641+2*np.pi*time_offset/(86164.09)
print ra

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='mollweide')

plt.pcolormesh(ra_mesh, dec_mesh, epsilon_abs, cmap = cm.magma)
plt.colorbar()
plt.xlabel('$ra$')
plt.ylabel('$dec$')
plt.show()

In [ ]:
#calculate eta
m1 = 1.48
m2 = 1.26
eta = (m1*m2)/(m1+m2)**2
mchirp = (m1*m2)**(3/5)/(m1+m2)**(1/5)
print eta
print mchirp